In [106]:
import glob                 #To get files from folder
import pandas as pd         #To read those files
import numpy as np          #To do mathematical operation
import os                   #Files manupulation 

from sklearn.naive_bayes import MultinomialNB                       #Model import from sklearn
from sklearn.feature_extraction.text import CountVectorizer          
from sklearn.model_selection import train_test_split                 
from sklearn.metrics import accuracy_score                           #How the classifire works/ How accruate it is!
import time                                                          #How much time it take to run
import re                                                            #Replacement
from sklearn.pipeline import Pipeline                                
from sklearn.feature_extraction.text import TfidfTransformer         
from sklearn.metrics import confusion_matrix                         #Model description with color 
import joblib                                                        


In [107]:
all_file = glob.glob("../songData/*.xlsx")
len(all_file)

21

In [108]:
all_file

g = all_file[2]
g.split('\\')[1][:-5]

'songType10'

In [109]:
names = []
count = []
clusters = []
data = []

for each in all_file:
    temp = pd.read_excel(each)
    
    
    count.append(len(temp))
    names.append(each.split('/')[1][:-5])
    clusters.append(temp['songType'])
    
    
    data.append(temp)
    
cluster_ids =clusters    

In [110]:
data = pd.concat(data)
data = data.sample(frac =1)

data

,Unnamed: 0,lyrics,songType
670,3956,"বুকের বক্রতা বুকেই রাখতাম, সময় উজাড় করে দুঃস...",13
57,3837,গান গাই আমার মনরে বুঝাই মন থাকে পাগলপারা আর কি...,11
53,1188,শীতের হাওয়ার লাগল নাচন আম্‌লকির এই ডালে ডালে। ...,17
326,1761,আজি মম মন চাহে জীবনবন্ধুরে সেই জনমে মরণে নিত্য...,17
21,96,"ছেলে= কত ভালো লাগে এইদিন এই পৃথিবী, যেন পাখীদে...",3
...,...,...,...
358,2250,জানতাম যদি শুভংকরের ফাকি আমার ঘরে পড়তো না ভাই ...,3
683,3712,দুঃখ ভালোবেসে প্রেমের খেলা খেলতে হয়। আগুনে মন ...,3
41,3434,সময় গেলে সাধন হবে না দিন থাকতে দ্বীনের সাধন কে...,19
402,1660,"নবাব নবাবী করে, নেতা নেতা-গিরি ট্রেনে ট্রেনে গ...",1


In [111]:
data['lyrics']

670    বুকের বক্রতা বুকেই রাখতাম, সময় উজাড় করে দুঃস...
57     গান গাই আমার মনরে বুঝাই মন থাকে পাগলপারা আর কি...
53     শীতের হাওয়ার লাগল নাচন আম্‌লকির এই ডালে ডালে। ...
326    আজি মম মন চাহে জীবনবন্ধুরে সেই জনমে মরণে নিত্য...
21     ছেলে= কত ভালো লাগে এইদিন এই পৃথিবী, যেন পাখীদে...
                             ...                        
358    জানতাম যদি শুভংকরের ফাকি আমার ঘরে পড়তো না ভাই ...
683    দুঃখ ভালোবেসে প্রেমের খেলা খেলতে হয়। আগুনে মন ...
41     সময় গেলে সাধন হবে না দিন থাকতে দ্বীনের সাধন কে...
402    নবাব নবাবী করে, নেতা নেতা-গিরি ট্রেনে ট্রেনে গ...
643    বিদেশ ঘুরে দেশে এলে দশ গেরামের সোনার ছেলে ভাই ...
Name: lyrics, Length: 4105, dtype: object

In [112]:
#stop_words = [ 'আমি','না','তুমি']

In [113]:
#data['lyrics'] = data['lyrics'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [114]:
X_train, X_test, Y_train, Y_test = train_test_split(data['lyrics'],data['songType'],test_size=0.33, random_state=42 )

In [115]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import metrics

total_data_count = len(data)
per_class_counts = []
unique_classes = np.unique(data['songType'].values)
class_count_mean = np.mean(count)

############### classifier setup ################

from sklearn.linear_model import SGDClassifier
text_clf_svm = Pipeline([
    
    ('vect', CountVectorizer(stop_words=stop_words,
                            analyzer="word", 
                            lowercase=False,
                            token_pattern="[\S]*",
                            tokenizer=None,
                            ngram_range=(1, 3),
                            preprocessor=None)),                 
    ('tfidf', TfidfTransformer()),
    ('clf-svm', SGDClassifier(loss = 'modified_huber', penalty='l2', max_iter=100)),
                                                                        
])
classifier = text_clf_svm.fit(list(X_train), list(Y_train))


cv = ShuffleSplit(n_splits=5, test_size=0.3, random_state=0)
score = cross_val_score(text_clf_svm, list(X_train), list(Y_train), cv=cv)

print("Cross Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))


predicted = classifier.predict(X_test)

C:\Users\MANSTEIN\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens [''] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Cross Accuracy: 0.52 (+/- 0.03)


In [116]:
joblib.dump(classifier, '../Model/sample_model.pickle')

['../Model/sample_model.pickle']

In [117]:
print("Cross Accuracy: %0.2f (+/- %0.2f)" % (score.mean(), score.std() * 2))
print(metrics.classification_report(Y_test, predicted))

Cross Accuracy: 0.52 (+/- 0.03)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         4
           1       0.49      0.51      0.50       342
           2       0.00      0.00      0.00         5
           3       0.38      0.59      0.47       226
           4       1.00      0.14      0.25         7
           5       0.67      0.12      0.21        16
           6       0.60      0.43      0.50        35
           7       0.70      0.41      0.51        64
           8       0.00      0.00      0.00         3
           9       0.00      0.00      0.00        17
          10       0.50      0.07      0.12        15
          11       0.12      0.05      0.07        20
          12       0.23      0.14      0.17        37
          13       0.53      0.52      0.53       239
          14       0.75      0.17      0.27        18
          16       0.00      0.00      0.00         2
          17       0.85      0.91      0.88      

C:\Users\MANSTEIN\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [118]:
import pandas as pd
confusion_matrix = list(metrics.confusion_matrix(Y_test, predicted))

In [119]:
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
array = confusion_matrix
df_cm = pd.DataFrame(array, index = [i for i in names],
              columns = [i for i in names])
df_norm_col=(df_cm-df_cm.mean())/df_cm.std()
#df_norm_col=df_cm-df_cm.mean()
plt.figure(figsize = (10,7))
sn.heatmap(df_norm_col, annot=True,cmap='viridis')

ValueError: 21 columns passed, passed data had 20 columns